## Day 3 - Conversation AI - aka Chatbot

In [1]:
import os
import ollama
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', '')
client = ollama.Client(host=os.getenv('OLLAMA_HOST', 'http://localhost:11434'))

In [7]:
openai = OpenAI()
gpt_model = 'gpt-4o-mini'

In [8]:
system_message = "You are helpful assistant"


### Reminder of the structure of prompt messages to OpenAI

[</br>
    {'role' : 'system', 'content': 'system messages here'},</br>
    {'role' : 'user', 'content' : first user prompt here'},</br>
    {'role' : 'assistant', 'content' : 'the assistant response'},</br>
    {'role' : 'user', 'content' : 'the new user prompt'}</br>
]</br>

we will write a function chat(message, history)</br>
where: message is the prompt to use history is a list of pairs of user message with assistant</br>
[</br>
    ['user said this', 'assistant replied']</br>
    ['then user said this', 'assistance replied again']</br>

]</br>
</br>
we will convert this history into the prompt style for OpenAI, then call OpenAI


In [ ]:
def gpt_chat(message, history):
    messages = [{'role': "system", 'content' : system_message}]
    for user_message, assitant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assitant_message})
    
    messages.append({'role': 'user', 'content': message})

    print('History is')
    print(history)
    print('and messages is')
    print(messages)

    stream = openai.chat.completions.create(
        model = gpt_model,
        messages = messages,
        stream = True
    )
    response = ''
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

    

In [18]:
def llama_chat(message, history):
    messages = [{'role': "system", 'content' : system_message}]
    for user_message, assitant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assitant_message})
    
    messages.append({'role': 'user', 'content': message})

    print('History is')
    print(history)
    print('and messages is')
    print(messages)

    response = client.chat(
        model = 'llama3.2',
        messages = messages,
        stream = True 
    )
    
    full_response = ''
    for chunk in response:
        if 'content' in chunk['message']:
            full_response += chunk['message']['content']
            yield full_response

In [ ]:
gr.ChatInterface(
    fn = gpt_chat
).launch()

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


History is
[]
and messages is
[{'role': 'system', 'content': 'You are helpful assistant'}, {'role': 'user', 'content': 'hello'}]
History is
[['hello', 'Hello! How can I assist you today?']]
and messages is
[{'role': 'system', 'content': 'You are helpful assistant'}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'I want to buy a car for my famlity'}]
History is
[['hello', 'Hello! How can I assist you today?'], ['I want to buy a car for my famlity', "That’s great! Buying a car for your family is an important decision. Here are some factors to consider when choosing a vehicle:\n\n1. **Budget**: What is your price range? Don’t forget to consider insurance, taxes, maintenance, and fuel costs.\n\n2. **Size and Capacity**: How many passengers do you need to accommodate? Consider if you need a compact car, sedan, SUV, or minivan.\n\n3. **Safety**: Look into the safety ratings and features of the car. Fa

In [ ]:
gr.ChatInterface(
    fn = llama_chat
).launch()

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


History is
[]
and messages is
[{'role': 'system', 'content': 'You are helpful assistant'}, {'role': 'user', 'content': 'hello'}]
History is
[['hello', "Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"]]
and messages is
[{'role': 'system', 'content': 'You are helpful assistant'}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': "Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"}, {'role': 'user', 'content': 'I want to see a good sight, can you recommand it in Newyork'}]
